In [9]:
!pip install --upgrade pip
!pip install -r local-requirements.txt

# make sure updates to the python modules are imported
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from sagemaker.s3 import S3Uploader
import sagemaker
import boto3
import string
import json

# Upload the data from the local dataset folder to the default bucket
output_bucket = sagemaker.Session().default_bucket()
        
local_data_file = "../datasets/template.json"
data_s3_location = f"s3://{output_bucket}/llm-evaluation-at-scale-example"
training_s3_location = f"{data_s3_location}/train_dataset"
validation_s3_location = f"{data_s3_location}/val_dataset"

print(f"Default data location: {data_s3_location}")
print(f"Default train location: {training_s3_location}")
print(f"Default data location: {validation_s3_location}")

S3Uploader.upload("../datasets/evaluation_dataset_trivia.jsonl", data_s3_location)
S3Uploader.upload("../datasets/train_dataset_trivia.jsonl", training_s3_location)
S3Uploader.upload("../datasets/template.json", training_s3_location)
S3Uploader.upload("../datasets/validation_dataset_trivia.jsonl", validation_s3_location)
S3Uploader.upload("../datasets/template.json", validation_s3_location)

Default data location: s3://sagemaker-us-east-1-221095015993/llm-evaluation-at-scale-example
Default train location: s3://sagemaker-us-east-1-221095015993/llm-evaluation-at-scale-example/train_dataset
Default data location: s3://sagemaker-us-east-1-221095015993/llm-evaluation-at-scale-example/val_dataset


's3://sagemaker-us-east-1-221095015993/llm-evaluation-at-scale-example/val_dataset/template.json'

In [12]:
# Optional Step - upload addition data for further testing via the Studio UI
data_s3_location_additional = f"s3://{output_bucket}/llm-evaluation-ui-example"

S3Uploader.upload("../datasets/other_datasets/workshop/evaluation_dataset_small.jsonl", data_s3_location_additional)
S3Uploader.upload("../datasets/other_datasets/workshop/evaluation_dataset_human.jsonl", data_s3_location_additional)

# Need for Amazon Clarify UI
def attach_cors_to_bucket(bucket_name):    
    s3 = boto3.client('s3')
  
    try:
        response = s3.put_bucket_cors(Bucket = bucket_name, 
                                      CORSConfiguration = {
                                            'CORSRules' : [
                                                {
                                                    'ID' : bucket_name + 'cors',
                                                    'AllowedHeaders' : [ '*' ],
                                                    'AllowedMethods' : [ 'PUT', 'GET', 'POST', 'DELETE', 'HEAD' ],
                                                    'AllowedOrigins' : [ '*' ],
                                                    'ExposeHeaders' :  [ 'ETag', 'x-amz-delete-marker', 'x-amz-server-side-encryption',
                                                                         'x-amz-request-id','x-amz-version-id','x-amz-id-2']
                                                }
                                            ]
                                        })
    except ClientError as e:
            return None
    return response

attach_cors_to_bucket(output_bucket)

{'ResponseMetadata': {'RequestId': '0R7CXX4P5A0EH9AQ',
  'HostId': 'cV+OGwj4rIN0vkqazZ2gcmtq2K8T2JjclhAsO5kBVnUuReps0LoIOLpmZc+6O75XNPCcKHFWbC9GUlzeCYzRd2aKQFMTBOOs88zI2xBtUFM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cV+OGwj4rIN0vkqazZ2gcmtq2K8T2JjclhAsO5kBVnUuReps0LoIOLpmZc+6O75XNPCcKHFWbC9GUlzeCYzRd2aKQFMTBOOs88zI2xBtUFM=',
   'x-amz-request-id': '0R7CXX4P5A0EH9AQ',
   'date': 'Fri, 19 Jan 2024 10:49:37 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [ ]:
!python pipeline.py --config pipeline_config.yaml

In [ ]:
!python pipeline_scale.py --config pipeline_scale_config.yaml

In [ ]:
!python pipeline_finetuning.py --config pipeline_finetuning_config.yaml

In [ ]:
!python pipeline_scale.py --config pipeline_scale_hybrid_config.yaml